# TensorFlow Recommenders: Quickstart

## Import TFRS

In [1]:
!pip3 install -q tensorflow-recommenders
!pip3 install -q --upgrade tensorflow-datasets

In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Read the data

In [3]:
ratings = tfds.load("movielens/100k-ratings", split="train")

movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [5]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

## Define a model

In [6]:
class MovieLensModel(tfrs.Model):

  def __init__(self,
               user_model: tf.keras.Model,
               movie_model: tf.keras.Model,
               task: tfrs.tasks.Retrieval):
    super().__init__()
  
    self.user_model = user_model
    self.movie_model = movie_model

    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [8]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)                                  
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)                                   
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))

## Fit and evaluate it.

In [10]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096), epochs=3)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index(movies.batch(100).map(model.movie_model), movies)

_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 29s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0042 - factorized_top_k/top_10_categorical_accuracy: 0.0102 - factorized_top_k/top_50_categorical_accuracy: 0.0725 - factorized_top_k/top_100_categorical_accuracy: 0.1557 - loss: 33246.6996 - regularization_loss: 0.0000e+00 - total_loss: 33246.6996
Epoch 2/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 5.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0079 - factorized_top_k/top_10_categorical_accuracy: 0.0206 - factorized_top_k/top_50_categorical_accuracy: 0.1358 - factorized_top_k/top_100_categorical_accuracy: 0.2572 - loss: 30590.3354 - regularization_loss: 0.0000e+00 - total_loss: 30590.3354
Epoch 3/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 5.7000e-04 - factorized_top_k/top_5_categorical_accu